# Protecting against Cross-Site Scripting

This notebook provides an example to showcase the methods we use to protect against Cross-Site Scripting (XSS).
In particular, to protect against it we escape some symbols in the JSON output and we add some extra headers which further ensure that the browser won't misidentify the content.

* **JSON serialiser.** We escape any HTML symbols on the output, using their unicode sequences instead.
* **Extra headers.** On every response, we set the `X-Content-Type-Options: nosniff;` header, which ensures that the browser won't try to guess the `Content-Type` from the content.

The organisation of this notebook is as follows:

1. Setup

    1. We will build a docker image of the `engine` to make sure it reflects the latest changes.  
    2. Then, we create a `k8s` cluster using `kind` installing `seldon-core`.  
    3. Lastly, we build a docker image of a dummy model which just repeats the input.  

2. Checking the response  

    1. Test with `curl` that the output escapes the HTML symbols.  
    2. Test with `curl` that the response includes the `X-Content-Type-Options` header.  

## Setup

Before showing a couple examples on how the output is modified to protect against XSS attacks, we will setup the environment.

### Build engine image

To make sure we are running the latest version of the engine, we will build a docker image from the current code.
Note that this requires a valid JDK installation.

In [3]:
!cd ../../../engine && make build_image

../proto/seldon_deployment.proto -> src/main/proto/seldon_deployment.proto
../proto/prediction.proto -> src/main/proto/prediction.proto
cp -vr ../proto/k8s/k8s.io src/main/proto
../proto/k8s/k8s.io -> src/main/proto/k8s.io
../proto/k8s/k8s.io/apis -> src/main/proto/k8s.io/apis
../proto/k8s/k8s.io/apis/meta -> src/main/proto/k8s.io/apis/meta
../proto/k8s/k8s.io/apis/meta/v1 -> src/main/proto/k8s.io/apis/meta/v1
../proto/k8s/k8s.io/api -> src/main/proto/k8s.io/api
../proto/k8s/k8s.io/api/core -> src/main/proto/k8s.io/api/core
../proto/k8s/k8s.io/api/core/v1 -> src/main/proto/k8s.io/api/core/v1
../proto/k8s/k8s.io/api/core/v1/generated.proto -> src/main/proto/k8s.io/api/core/v1/generated.proto
../proto/k8s/k8s.io/api/core/v1/generated.protobak -> src/main/proto/k8s.io/api/core/v1/generated.protobak
../proto/k8s/k8s.io/api/autoscaling -> src/main/proto/k8s.io/api/autoscaling
../proto/k8s/k8s.io/api/autoscaling/v2beta1 -> src/main/proto/k8s.io/api/autoscaling/v2beta1
../proto/k8s/k8s.io/api

16:49:18.574 [main] DEBUG org.springframework.test.context.support.AbstractContextLoader - Did not detect default resource location for test class [io.seldon.engine.predictors.RandomABTestUnitTest]: class path resource [io/seldon/engine/predictors/RandomABTestUnitTest-context.xml] does not exist
16:49:18.578 [main] DEBUG org.springframework.test.context.support.AbstractContextLoader - Did not detect default resource location for test class [io.seldon.engine.predictors.RandomABTestUnitTest]: class path resource [io/seldon/engine/predictors/RandomABTestUnitTestContext.groovy] does not exist
16:49:18.581 [main] INFO org.springframework.test.context.support.AbstractContextLoader - Could not detect default resource locations for test class [io.seldon.engine.predictors.RandomABTestUnitTest]: no resource found for suffixes {-context.xml, Context.groovy}.
16:49:18.588 [main] INFO org.springframework.test.context.support.AnnotationConfigContextLoaderUtils - Could not detect default configuratio

16:49:18.770 [main] DEBUG org.springframework.core.env.StandardEnvironment - Initialized StandardEnvironment with PropertySources [MapPropertySource@623446986 {name='systemProperties', properties={gopherProxySet=false, awt.toolkit=sun.lwawt.macosx.LWCToolkit, java.specification.version=11, sun.cpu.isalist=, sun.jnu.encoding=UTF-8, java.class.path=/Users/kaseyo/.m2/repository/org/apache/maven/surefire/surefire-booter/2.18.1/surefire-booter-2.18.1.jar:/Users/kaseyo/.m2/repository/org/apache/maven/surefire/surefire-api/2.18.1/surefire-api-2.18.1.jar:/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes:/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/classes:/Users/kaseyo/.m2/repository/org/ojalgo/ojalgo/47.0.0/ojalgo-47.0.0.jar:/Users/kaseyo/.m2/repository/org/springframework/boot/spring-boot-starter-test/1.5.17.RELEASE/spring-boot-starter-test-1.5.17.RELEASE.jar:/Users/kaseyo/.m2/repository/org/springframework/boot/spring-boot-test/1.5.17.RELEASE/spring-boot-test-1.5

16:49:18.871 [main] DEBUG org.springframework.core.io.support.PathMatchingResourcePatternResolver - Resolved classpath location [io/seldon/engine/predictors/] to resources [URL [file:/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors/], URL [file:/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/classes/io/seldon/engine/predictors/]]
16:49:18.872 [main] DEBUG org.springframework.core.io.support.PathMatchingResourcePatternResolver - Looking for matching resources in directory tree [/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors]
16:49:18.872 [main] DEBUG org.springframework.core.io.support.PathMatchingResourcePatternResolver - Searching directory [/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors] for files matching pattern [/Users/kaseyo/Seldon/seldon-core-mirror1/engine/target/test-classes/io/seldon/engine/predictors/*.class]
16:49:18.886 [ma

16:49:19.410 [main] INFO org.springframework.boot.test.context.SpringBootTestContextBootstrapper - Using TestExecutionListeners: [org.springframework.test.context.web.ServletTestExecutionListener@7aaad0, org.springframework.test.context.support.DirtiesContextBeforeModesTestExecutionListener@eed890d, org.springframework.boot.test.mock.mockito.MockitoTestExecutionListener@10f477e2, org.springframework.boot.test.autoconfigure.SpringBootDependencyInjectionTestExecutionListener@6097fca9, org.springframework.test.context.support.DirtiesContextTestExecutionListener@35eee641, org.springframework.test.context.transaction.TransactionalTestExecutionListener@5729b410, org.springframework.test.context.jdbc.SqlScriptsTestExecutionListener@64518270, org.springframework.boot.test.mock.mockito.ResetMocksTestExecutionListener@3b7c58e7, org.springframework.boot.test.autoconfigure.restdocs.RestDocsTestExecutionListener@79627d27, org.springframework.boot.test.autoconfigure.web.client.MockRestServiceServerR

...

{
  "meta": {
    "puid": "lopang8qtr367fn32k6bcqnfah",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "lopang8qtr367fn32k6bcqnfah",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "lopang8qtr367fn32k6bcqnfah",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{


{
  "meta": {
    "puid": "kns60re7lgusmn74h43vu6mumf",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "kns60re7lgusmn74h43vu6mumf",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "kns60re7lgusmn74h43vu6mumf",
    "tags": {
    },
    "routing": {
      "abtest": 0
    },
    "requestPath": {
      "abtest": "",
      "model1": "seldonio/model1:0.6"
    },
    "metrics": [{


{
  "meta": {
    "puid": "f8joqoa30uh2gfsvljfucraai4",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "f8joqoa30uh2gfsvljfucraai4",
    "tags": {
    },
    "routing": {
      "abtest": 1
    },
    "requestPath": {
      "abtest": "",
      "model2": "seldonio/model2:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "f8joqoa30uh2gfsvljfucraai4",
    "tags": {
    },
    "routing": {
      "abtest": 1
    },
    "requestPath": {
      "abtest": "",
      "model2": "seldonio/model2:0.6"
    },
    "metrics": [{


{
  "meta": {
    "puid": "q928sldict5gntop6jtlma0337",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
    },
    "metrics": []
  }
}
{
  "meta": {
    "puid": "q928sldict5gntop6jtlma0337",
    "tags": {
    },
    "routing": {
      "abtest": 1
    },
    "requestPath": {
      "abtest": "",
      "model2": "seldonio/model2:0.6"
    },
    "metrics": [{
      "key": "mycounter",
      "type": "COUNTER",
      "value": 1.0,
      "tags": {
        "mytag1": "mytagval1"
      }
    }, {
      "key": "mygauge",
      "type": "GAUGE",
      "value": 22.0,
      "tags": {
      }
    }, {
      "key": "mytimer",
      "type": "TIMER",
      "value": 1.0,
      "tags": {
      }
    }]
  },
  "data": {
    "names": [],
    "ndarray": [[1.0, 2.0]]
  }
}
{
  "meta": {
    "puid": "q928sldict5gntop6jtlma0337",
    "tags": {
    },
    "routing": {
      "abtest": 1
    },
    "requestPath": {
      "abtest": "",
      "model2": "seldonio/model2:0.6"
    },
    "metrics": [{


[WARNING] POM for dependency org.mockito:mockito-core has a license URL that returns file not found: http://github.com/mockito/mockito/blob/master/LICENSE
[WARNING] POM for dependency com.microsoft.azure:adal4j has an invalid license URL: null
[WARNING] POM for dependency jline:jline has an invalid license URL: LICENSE.txt
[WARNING] POM for dependency com.nimbusds:oauth2-oidc-sdk has a license URL that returns file not found: http://www.apache.org/licenses/LICENSE-2.0.html‎
[WARNING] No license information available for: org.apache.zookeeper:zookeeper
[WARNING] Unable to retrieve license for dependency: javax.activation:activation
[WARNING] https://glassfish.dev.java.net/public/CDDLv1.0.html
[WARNING] No subject alternative DNS name matching glassfish.dev.java.net found.
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  01:03 min
[INFO] 

### Create k8s cluster

Firstly, we will create a cluster using [kind](https://kind.sigs.k8s.io).

In [2]:
!kind create cluster
!export KUBECONFIG="$(kind get kubeconfig-path --name=kind)"

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.15.3) 🖼 
 ✓ Preparing nodes 📦 
 ✓ Creating kubeadm config 📜 
 ✓ Starting control-plane 🕹️ 
 ✓ Installing CNI 🔌 
 ✓ Installing StorageClass 💾 
Cluster creation complete. You can now use the cluster with:

export KUBECONFIG="$(kind get kubeconfig-path --name="kind")"
kubectl cluster-info


We then install Helm and a corresponding service account.

In [3]:
!helm init --history-max 200
!kubectl rollout status deploy/tiller-deploy -n kube-system
!kubectl create serviceaccount --namespace kube-system tiller
!kubectl create clusterrolebinding tiller-cluster-rule --clusterrole=cluster-admin --serviceaccount=kube-system:tiller
!kubectl patch deploy --namespace kube-system tiller-deploy -p '{"spec":{"template":{"spec":{"serviceAccount":"tiller"}}}}'

$HELM_HOME has been configured at /Users/kaseyo/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Waiting for deployment spec update to be observed...
Waiting for deployment spec update to be observed...
Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out
serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller-cluster-rule created
deployment.extensions/tiller-deploy patched


Next, **before installing `seldon-core`**, we load the engine image we have just built above into the cluster.

In [5]:
!kind load docker-image seldonio/engine:0.3.1

We can now install `seldon-core` on the new cluster, making sure that it uses the engine image local to the nodes.

In [7]:
!helm install \
    ../helm-charts/seldon-core-operator \
    --name seldon-core \
    --namespace seldon-system \
    --set engine.image.pullPolicy=Never \
    --set usagemetrics.enabled=true \
    --set ambassador.enabled=true
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

NAME:   seldon-core
LAST DEPLOYED: Wed Sep 18 11:17:36 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRole
NAME                          AGE
seldon-operator-manager-role  3s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  3s

==> v1/ConfigMap
NAME           DATA  AGE
seldon-config  1     3s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0  0/1    ContainerCreating  0         1s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     3s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.109.215.4  <none>       443/TCP  2s
webhook-server-service                      ClusterIP  10.103.10.81  <none>       443/TCP  2s

==> v1/ServiceAccount

Finally, we install `ambassador` which will allow us to reach the Seldon engine in the cluster.

In [8]:
!helm install stable/ambassador --name ambassador --set crds.keep=false
!kubectl rollout status deployment.apps/ambassador

NAME:   ambassador
LAST DEPLOYED: Wed Sep 18 11:18:14 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Deployment
NAME        READY  UP-TO-DATE  AVAILABLE  AGE
ambassador  0/3    3           0          3s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-5784b5cb9d-2jr8x  0/1    ContainerCreating  0         2s
ambassador-5784b5cb9d-2tq48  0/1    ContainerCreating  0         2s
ambassador-5784b5cb9d-j4k85  0/1    ContainerCreating  0         2s

==> v1/Service
NAME              TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)                     AGE
ambassador        LoadBalancer  10.110.97.64   <pending>    80:30872/TCP,443:32557/TCP  3s
ambassador-admin  ClusterIP     10.105.105.80  <none>       8877/TCP                    3s

==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        3s

==> v1beta1/ClusterRole
NAME             AGE
ambassador       3s
ambassador-crds  3s

==> v1beta1/ClusterRoleBinding
NAME      

### Forward port

Once the cluster has been created, we need to allow access from the outside to the `ambassador` gateway.
One way to do this is to use the `kubectl port-forward` command.
In particular, we will forward port `8003` of our local host to the cluster's gateway.

This command needs to run constantly on the background, so **please make sure you run it on a separate terminal**.

```bash
kubectl \
    port-forward \
    $(kubectl get pods \
        -n seldon-core -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') \
    8003:8080
```

### Dummy Model

To test how `seldon-core` processes the output to prevent XSS attacks we will use a dummy model which just replies with whatever input we send.
The code for this model can be seen below.

In [24]:
!pygmentize ./XSSModel.py

class XSSModel(object):
    """
    Dummy model which just returns its input back.
    """

    def predict(self, X, feature_names):
        return X


Firstly, we will build an appropiate image using `s2i`.
The name of this image will be `xss-model:0.1`.

In [25]:
!make build_image

s2i build . seldonio/seldon-core-s2i-python3:0.7 xss-model:0.1
error: Unable to load docker config: json: cannot unmarshal string into Go value of type docker.dockerConfig
---> Installing application source...
Build completed successfully


We are now ready to spin up a service running our model.
Note that before, we need to load the image into our `kind` cluster.

In [26]:
!kind load docker-image xss-model:0.1

In [27]:
!pygmentize ./xss-example.json

{
  "apiVersion": "machinelearning.seldon.io/v1alpha2",
  "kind": "SeldonDeployment",
  "metadata": {
    "labels": {
      "app": "seldon"
    },
    "name": "xss-example"
  },
  "spec": {
    "name": "xss-example",
    "predictors": [
      {
        "componentSpecs": [
          {
            "spec": {
              "containers": [
                {
                  "image": "xss-model:0.1",
                  "imagePullPolicy": "IfNotPresent",
                  "name": "xss-model"
                }
              ]
            }
          }
        ],
        "graph": {
          "children": [],
          "name": "xss-model",
          "endpoint": {
            "type": "REST"
          },
          "type": "MODEL"
        },
        "name": "xss-model",
        "replicas": 1
      }
    ]
  }
}


In [38]:
!kubectl apply -f ./xss-example.json

seldondeployment.machinelearning.seldon.io/xss-example created


To visualise what the model does and verify that everything is working we can make an example request using `curl`.
Note that, on the request we are passing a string field as `{"strData": "hello world"}`.
On the output, we receive the same field after being returned as-is by `XSSModel`.

In [40]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"strData": "hello world"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "3cf8scoctfk44ghs8vfpe61ovb",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "strData": "hello world"
}

## Checking the response

### JSON serialiser

To showcase the escaping of HTML characters in the JSON output, we will submit a HTML payload in our request.
Note that the output uses the corresponding unicode value, instead of the sensible character.
This helps to avoid undesired behaviour when the output could be mis-interpreted as HTML.

In [43]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"strData": "<div class=\"box\">This is a div</div>"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "vphfe82nuh45msp702pemal55j",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "strData": "\u003cdiv class\u003d\"box\"\u003eThis is a div\u003c/div\u003e"
}

We can also verify that the output for anything else remains untouched.

In [4]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"strData": "Not HTML!"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "gj4qo7sbtn5928cqhjuqbfud4m",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "strData": "Not HTML!"
}

In [2]:
!curl \
    -X POST \
    -H 'Content-Type: application/json' \
    -d '{"data": {"ndarray": [0, 1, 2, 3, 4]}}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions

{
  "meta": {
    "puid": "tiu0rnm09hrd21gvgap4o2inqc",
    "tags": {
    },
    "routing": {
    },
    "requestPath": {
      "xss-model": "xss-model:0.1"
    },
    "metrics": []
  },
  "data": {
    "names": [],
    "ndarray": [0.0, 1.0, 2.0, 3.0, 4.0]
  }
}

### Extra headers

Similarly, we can show the response headers, to see that the `X-Content-Type-Options` header is included in the response.
This header will avoid the browser trying to infer the content type and trusting the already sent `Content-Type` header instead.

In [8]:
!curl \
    -X POST \
    -sD - -o /dev/null \
    -H 'Content-Type: application/json' \
    -d '{"strData": "<div class=\"box\">This is a div</div>"}' \
    localhost:8003/seldon/default/xss-example/api/v0.1/predictions